<a href="https://colab.research.google.com/github/OpenXRF/lead-screening/blob/main/Figure%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenXRF_logo_red.svg

# **Towards low-cost lead screening with transmission XRF**
---


[Arxiv citation to come]


*   Project website: [openxrf.org](https://openxrf.org/)
*   GitHub repository: [github.com/OpenXRF/lead-screening](https://github.com/OpenXRF/lead-screening/)


---


In [ ]:
#@title Install packages { display-mode: "form" }
# @markdown Python packages for data processing and visualization

!pip install -q gdown pandas numpy matplotlib plotly
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from time import time

print("✓ All packages installed successfully!")

✓ All packages installed successfully!


# **Figure 1c:** $^{241}$Am spectrum


---



- Sample from an ionizing smoke detector
- Activity likely ~37 kBq (standard)
- Recorded with a silicon drift detector

In [ ]:
#@title Load and visualize the smoke detector ²⁴¹Am spectrum { display-mode: "form" }

import pandas as pd
import plotly.graph_objects as go

# Download the spectrum data
id_spectrum = "19AoPLWBzCrCgUB44ssf7eQe2rxig3bkL"
gdown.download(id=id_spectrum,
               output="data_Am241_fluence.csv",
               quiet=False)

# Load the CSV file
spectrum_data = pd.read_csv("data_Am241_fluence.csv")

# Get the correct column names (assuming first column is energy, second is fluence)
energy_col = spectrum_data.columns[0]
fluence_col = spectrum_data.columns[1]


print(f"✓ Loaded {len(spectrum_data)} data points")
print(f"  Energy range: {spectrum_data[energy_col].min():.1f} - {spectrum_data[energy_col].max():.1f} keV")

# Create the plot
fig = go.Figure()

# Add spectrum trace (using the blue color from the background)
fig.add_trace(
    go.Scatter(
        x=spectrum_data[energy_col],
        y=spectrum_data[fluence_col],
        fill='tozeroy',
        fillcolor="rgba(0,114,189,0.8)",
        line=dict(color="rgb(0,114,189)", width=1.5),
        name="²⁴¹Am spectrum",
        showlegend=True,
    )
)

# Calculate ylim based on data
y_max = spectrum_data[fluence_col].max()
ylim = (0, y_max * 1.05)  # Add 5% padding at the top

# Update layout to match the style
fig.update_layout(
    xaxis_title="Energy (keV)",
    yaxis_title="Fluence (photons/keV/s)",
    font=dict(size=12),
    height=500,
    width=800,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=80, l=80, r=40, b=60),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

# Update axes
fig.update_xaxes(
    showgrid=False,        # Remove grid lines
    showticklabels=True,   # Keep tick labels (this is default, but explicit here)
    ticks="outside",       # Show tick marks outside the plot
    showline=True,         # Show axis line
    linewidth=1,           # Axis line thickness
    linecolor='black',     # Axis line color
    mirror=False,          # Don't mirror to top
    zeroline=False
)
fig.update_yaxes(
    range=ylim,
    showgrid=False,        # Remove grid lines
    showticklabels=True,   # Keep tick labels (this is default, but explicit here)
    ticks="outside",       # Show tick marks outside the plot
    showline=True,         # Show axis line
    linewidth=1,           # Axis line thickness
    linecolor='black',     # Axis line color
    mirror=False,          # Don't mirror to right
    zeroline=False
)

fig.show()

Downloading...
From: https://drive.google.com/uc?id=19AoPLWBzCrCgUB44ssf7eQe2rxig3bkL
To: /content/data_Am241_fluence.csv
100%|██████████| 19.8k/19.8k [00:00<00:00, 12.9MB/s]

✓ Loaded 1024 data points
  Energy range: 0.0 - 71.6 keV


# **Figure 1d:** Comparing backscatter and transmission geometries


---


* Monte Carlo simulations (Geant 4)
* Soil ([NIST, SRM 2587](https://tsapps.nist.gov/srmext/certificates/2587.pdf)) with 100 ppm Pb
* Distances found in the paper


In [ ]:
#@title 1) Load simulated data

start_time = time()

id_backscatter = "1XobbLRaWrPsWE1FHDcAFQWGz-CpIreDW";
id_transmission = "1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43";

gdown.download(id=id_backscatter,
               output="data_backscatter_realistic_108particles_100ppm.csv",
               quiet=False)

gdown.download(id=id_transmission,
               output="data_transmission_108particles_100ppm.csv",
               quiet=False)

data_b = pd.read_csv("data_backscatter_realistic_108particles_100ppm.csv")
data_t = pd.read_csv("data_transmission_108particles_100ppm.csv")

print(f"✓ Data loaded successfully (took {time() - start_time:.1f} s)")
print(f"  - Backscatter data: {len(data_b):,} photon events")
print(f"  - Transmission data: {len(data_t):,} photon events")

📥 Loading simulated data...


Downloading...
From: https://drive.google.com/uc?id=1XobbLRaWrPsWE1FHDcAFQWGz-CpIreDW
To: /content/data_backscatter_realistic_108particles_100ppm.csv
100%|██████████| 23.1M/23.1M [00:00<00:00, 136MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43
From (redirected): https://drive.google.com/uc?id=1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43&confirm=t&uuid=1a74a08d-86ed-41a2-89aa-0dcacbc069fb
To: /content/data_transmission_108particles_100ppm.csv
100%|██████████| 2.16G/2.16G [00:23<00:00, 93.2MB/s]


✓ Data loaded successfully (took 93.2 s)
  - Backscatter data: 161,253 photon events
  - Transmission data: 18,023,305 photon events


In [ ]:
#@title 2) Adjust experimental parameters and visualize results { display-mode: "form" }

# @markdown **Energy resolution**: Detector energy resolution (FWHM) [keV]
FWHM_keV = 0.15 #@param {name:"s", type:"slider", min:0.15, max:1.0, step:0.05}
# @markdown **Energy bin width**: Histogram bin size [eV]
binwidth_eV = 40 #@param {type:"slider", min:30, max:150, step:10}
# @markdown **Detector area**: Active detector area [mm²] (assuming 100% detection efficiency)
detector_area_mm2 = 50 #@param {type:"slider", min:10, max:50, step:5}
# @markdown **Exposure time**: Measurement duration [hours]
exposure_hours = 10 #@param {type:"slider", min:1, max:10, step:1}

print("=" * 60)
print("Experimental parameters")
print("=" * 60)
print(f"Detector energy resolution (FWHM): {FWHM_keV:.2f} keV")
print(f"Detector energy bin bidth: {binwidth_eV} eV ({binwidth_eV/1000:.3f} keV)")
print(f"Detector area: {detector_area_mm2} mm²")
print(f"Exposure time: {exposure_hours} hours")
print("=" * 60)

# Convert bin width to keV
binwidth = binwidth_eV / 1000.0

from google.colab import output
output.no_vertical_scroll()

# ============================================================
# 1. ADD ENERGY RESOLUTION
# ============================================================
print("\n📊 Adding energy resolution...")
sigma = FWHM_keV / 2.355
rng = np.random.default_rng(42)  # Fixed seed for reproducibility

# Add Gaussian noise to simulate detector resolution
data_b["E"] = data_b["Energy(MeV)"] * 1000.0 + sigma * rng.standard_normal(len(data_b))
data_t["E"] = data_t["Energy(MeV)"] * 1000.0 + sigma * rng.standard_normal(len(data_t))

# ============================================================
# 2. FILTER EVENTS
# ============================================================
print("🔍 Filtering events based on exposure time and detector area...")

# Calculate photon fluence for exposure time
fluence = 995 * np.pi * 2 * (1 - np.cos(np.pi / 4))  # photons/s in 90° cone
N = exposure_hours * 60 * 60 * fluence  # total photons

# Calculate detector radius from area
detector_radius = np.sqrt(detector_area_mm2 / np.pi)

# Calculate hit radius for each event
for df in [data_b, data_t]:
    df["r_hit"] = np.sqrt(df["x(mm)"] ** 2 + df["y(mm)"] ** 2)

# Apply filters
mask_b = (
    (data_b["EventID"] < N) &
    (data_b["Particle"] == "gamma") &
    (data_b["r_hit"] <= detector_radius)
)
mask_t = (
    (data_t["EventID"] < N) &
    (data_t["Particle"] == "gamma") &
    (data_t["r_hit"] <= detector_radius)
)

data_b_filtered = data_b[mask_b]
data_t_filtered = data_t[mask_t]

# Separate primary and secondary events
primary_b = data_b_filtered[data_b_filtered["Type"] == "Primary"]
secondary_b = data_b_filtered[data_b_filtered["Type"] == "Secondary"]
primary_t = data_t_filtered[data_t_filtered["Type"] == "Primary"]
secondary_t = data_t_filtered[data_t_filtered["Type"] == "Secondary"]

print(f"  - Backscatter: {len(data_b_filtered):,} events detected")
print(f"  - Transmission: {len(data_t_filtered):,} events detected")

# ============================================================
# 3. CREATE HISTOGRAMS
# ============================================================
print("📈 Creating energy histograms...")

E = np.arange(0, 75 + binwidth, binwidth)

def hist(data):
    return np.histogram(data["E"], bins=np.append(E, E[-1] + binwidth))[0]

b = {
    "primary": hist(primary_b),
    "secondary": hist(secondary_b),
}
t = {
    "primary": hist(primary_t),
    "secondary": hist(secondary_t),
}

# ============================================================
# 4. CALCULATE SNR
# ============================================================
print("🎯 Calculating Signal-to-Noise Ratios...")

# Define peak windows
peak = {
    "alpha": {"E": 10.52},  # Pb Lα
    "beta": {"E": 12.65},   # Pb Lβ
}
peak_window = FWHM_keV

for name in ["alpha", "beta"]:
    mask = (E >= peak[name]["E"] - peak_window) & (E <= peak[name]["E"] + peak_window)
    peak[name]["window"] = mask

def calc_snr(hist_primary, hist_secondary, peak_mask):
    N_background = np.sum(hist_primary[peak_mask])
    N_signal = np.sum(hist_secondary[peak_mask])
    SNR = N_signal / np.sqrt(N_signal + N_background) if (N_signal + N_background) > 0 else 0
    return N_background, N_signal, SNR

# Calculate for both geometries
for geom, d in zip(["b", "t"], [b, t]):
    for name in ["alpha", "beta"]:
        Np, Ns, SNR = calc_snr(d["primary"], d["secondary"], peak[name]["window"])
        d[name] = {"primary": Np, "secondary": Ns, "SNR": SNR}

# ============================================================
# RESULTS SUMMARY
# ============================================================
print("\n" + "=" * 60)
print("Results summary")
print("=" * 60)
print("\nBackscatter geometry:")
print(f"  Pb Lα: SNR = {b['alpha']['SNR']:.1f} (Signal: {b['alpha']['secondary']:.0f}, Background: {b['alpha']['primary']:.0f})")
print(f"  Pb Lβ: SNR = {b['beta']['SNR']:.1f} (Signal: {b['beta']['secondary']:.0f}, Background: {b['beta']['primary']:.0f})")

print("\nTransmission geometry:")
print(f"  Pb Lα: SNR = {t['alpha']['SNR']:.1f} (Signal: {t['alpha']['secondary']:.0f}, Background: {t['alpha']['primary']:.0f})")
print(f"  Pb Lβ: SNR = {t['beta']['SNR']:.1f} (Signal: {t['beta']['secondary']:.0f}, Background: {t['beta']['primary']:.0f})")
print("=" * 60)

print("\n✓ Processing complete!")

print("🎨 Generating visualization...")

# Geometry parameters
sample_thickness_transmission = 2.5  # mm
sample_thickness_backscatter = 25.0  # mm
sample_width = 20.0  # mm
source_to_surface = 2.0
detector_to_surface = 3.0
detector_height = 0.2

# Calculate detector width based on area (approximate as rectangle with diameter ≈ 2*radius)
detector_width = 2 * np.sqrt(detector_area_mm2 / np.pi)

xLimGeom = [-10, 10]
yLimGeom = [-4, 7]

# Create subplot figure
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "<b>Backscatter Geometry</b>",
        "<b>Transmission Geometry</b>",
        "",
        "",
        "",
        "",
    ),
    horizontal_spacing=0.10,
    vertical_spacing=0.12,
    row_heights=[0.25, 0.375, 0.375],
    specs=[
        [{"type": "xy"}, {"type": "xy"}],
        [{"type": "xy"}, {"type": "xy"}],
        [{"type": "xy"}, {"type": "xy"}]
    ]
)

# ============================================================
# ROW 1: GEOMETRY DIAGRAMS
# ============================================================

# --- BACKSCATTER GEOMETRY ---
sample_y = sample_thickness_transmission - sample_thickness_backscatter

# Sample
fig.add_shape(
    type="rect",
    x0=-sample_width/2, x1=sample_width/2,
    y0=sample_y, y1=sample_y + sample_thickness_backscatter,
    fillcolor="rgb(230, 204, 179)",
    line=dict(color="rgba(0,0,0,0)"),
    row=1, col=1
)

# Source
source_x = -5.5
source_y = sample_y + sample_thickness_backscatter + source_to_surface
fig.add_trace(
    go.Scatter(
        x=[source_x], y=[source_y],
        mode="markers+text",
        marker=dict(size=12, color="rgb(255,140,0)", symbol="star"),
        text=["²⁴¹Am"], textposition="top center",
        showlegend=False
    ),
    row=1, col=1
)

# Detector
detector_x = -0.5
detector_y = sample_y + sample_thickness_backscatter + detector_to_surface
fig.add_shape(
    type="rect",
    x0=detector_x, x1=detector_x + detector_width,
    y0=detector_y, y1=detector_y + detector_height,
    fillcolor="rgb(70,130,180)",
    line=dict(color="black", width=2),
    row=1, col=1
)
fig.add_annotation(
    x=detector_x + detector_width/2, y=detector_y + detector_height + 0.5,
    text=f"Detector ({detector_area_mm2} mm²)", showarrow=False, font=dict(size=9),
    row=1, col=1
)

# X-ray cone (original blue color)
cone_len = 30
angles = np.deg2rad([-5, -95])
cone_x = [source_x, source_x + cone_len * np.cos(angles[0]),
          source_x + cone_len * np.cos(angles[1]), source_x]
cone_y = [source_y, source_y + cone_len * np.sin(angles[0]),
          source_y + cone_len * np.sin(angles[1]), source_y]
fig.add_trace(
    go.Scatter(
        x=cone_x, y=cone_y,
        fill="toself",
        fillcolor="rgba(77,179,255,0.2)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False
    ),
    row=1, col=1
)

# Add scale bar (1 mm)
scalebar_length = 1.0  # mm
scalebar_x = 7.5
scalebar_y = -3.0
fig.add_shape(
    type="line",
    x0=scalebar_x, x1=scalebar_x + scalebar_length,
    y0=scalebar_y, y1=scalebar_y,
    line=dict(color="black", width=3),
    row=1, col=1
)
# Add vertical ticks to scale bar
tick_height = 0.2
fig.add_shape(type="line", x0=scalebar_x, x1=scalebar_x,
              y0=scalebar_y-tick_height/2, y1=scalebar_y+tick_height/2,
              line=dict(color="black", width=2), row=1, col=1)
fig.add_shape(type="line", x0=scalebar_x+scalebar_length, x1=scalebar_x+scalebar_length,
              y0=scalebar_y-tick_height/2, y1=scalebar_y+tick_height/2,
              line=dict(color="black", width=2), row=1, col=1)
fig.add_annotation(
    x=scalebar_x + scalebar_length/2, y=scalebar_y - 0.5,
    text="1 mm", showarrow=False, font=dict(size=9),
    row=1, col=1
)

fig.update_xaxes(
    range=xLimGeom,
    showticklabels=False,
    showgrid=False,
    zeroline=False,
    row=1,
    col=1)
fig.update_yaxes(range=yLimGeom,
                 scaleanchor="x",
                 scaleratio=1,
                 showticklabels=False,
                 showgrid=False,
                 zeroline=False,
                 row=1,
                 col=1)

# --- TRANSMISSION GEOMETRY ---
# Sample
fig.add_shape(
    type="rect",
    x0=-sample_width/2, x1=sample_width/2,
    y0=0, y1=sample_thickness_transmission,
    fillcolor="rgb(230, 204, 179)",
    line=dict(color="rgba(0,0,0,0)"),
    row=1, col=2
)

# Source
source_x_t = 0
source_y_t = sample_thickness_transmission + source_to_surface
fig.add_trace(
    go.Scatter(
        x=[source_x_t], y=[source_y_t],
        mode="markers+text",
        marker=dict(size=12, color="rgb(255,140,0)", symbol="star"),
        text=["²⁴¹Am"], textposition="top center",
        showlegend=False
    ),
    row=1, col=2
)

# Detector
detector_y_t = -detector_to_surface - detector_height
fig.add_shape(
    type="rect",
    x0=source_x_t - detector_width/2, x1=source_x_t + detector_width/2,
    y0=detector_y_t, y1=detector_y_t + detector_height,
    fillcolor="rgb(70,130,180)",
    line=dict(color="black", width=2),
    row=1, col=2
)
fig.add_annotation(
    x=source_x_t, y=detector_y_t - 0.5,
    text=f"Detector ({detector_area_mm2} mm²)", showarrow=False, font=dict(size=9),
    row=1, col=2
)

# X-ray cone (original blue color)
angles_t = np.deg2rad([-45, -135])
cone_x_t = [source_x_t, source_x_t + cone_len * np.cos(angles_t[0]),
            source_x_t + cone_len * np.cos(angles_t[1]), source_x_t]
cone_y_t = [source_y_t, source_y_t + cone_len * np.sin(angles_t[0]),
            source_y_t + cone_len * np.sin(angles_t[1]), source_y_t]
fig.add_trace(
    go.Scatter(
        x=cone_x_t, y=cone_y_t,
        fill="toself",
        fillcolor="rgba(77,179,255,0.2)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False
    ),
    row=1, col=2
)

# Add scale bar (1 mm)
fig.add_shape(
    type="line",
    x0=scalebar_x, x1=scalebar_x + scalebar_length,
    y0=scalebar_y, y1=scalebar_y,
    line=dict(color="black", width=3),
    row=1, col=2
)
# Add vertical ticks to scale bar
fig.add_shape(type="line", x0=scalebar_x, x1=scalebar_x,
              y0=scalebar_y-tick_height/2, y1=scalebar_y+tick_height/2,
              line=dict(color="black", width=2), row=1, col=2)
fig.add_shape(type="line", x0=scalebar_x+scalebar_length, x1=scalebar_x+scalebar_length,
              y0=scalebar_y-tick_height/2, y1=scalebar_y+tick_height/2,
              line=dict(color="black", width=2), row=1, col=2)
fig.add_annotation(
    x=scalebar_x + scalebar_length/2, y=scalebar_y - 0.5,
    text="1 mm", showarrow=False, font=dict(size=9),
    row=1, col=2
)

fig.update_xaxes(range=xLimGeom, showticklabels=False, showgrid=False, zeroline=False, row=1, col=2)
fig.update_yaxes(range=yLimGeom, scaleanchor="x", scaleratio=1, showticklabels=False, showgrid=False, zeroline=False, row=1, col=2)

# ============================================================
# HELPER FUNCTION FOR SPECTRA
# ============================================================

def add_spectrum(row, col, E, primary, secondary, peak, xlim, data, showlegend=False):
    """Add spectrum plot with signal and background"""

    # Signal (red)
    fig.add_trace(
        go.Scatter(
            x=E, y=primary + secondary,
            fill='tozeroy',
            fillcolor="rgba(201,24,31,0.6)",
            line=dict(color="rgb(201,24,31)", width=1),
            name="XRF signal",
            showlegend=showlegend,
        ),
        row=row, col=col
    )

    # Background (blue)
    fig.add_trace(
        go.Scatter(
            x=E, y=primary,
            fill='tozeroy',
            fillcolor="rgba(0,114,189,0.8)",
            line=dict(color="rgb(0,114,189)", width=1),
            name="Background",
            showlegend=showlegend,
        ),
        row=row, col=col
    )

    # Peak window markers
    for name in ["alpha", "beta"]:
        fig.add_vline(
            x=peak[name]["E"] - peak_window,
            line_dash="dash", line_color="gray", line_width=1,
            row=row, col=col
        )
        fig.add_vline(
            x=peak[name]["E"] + peak_window,
            line_dash="dash", line_color="gray", line_width=1,
            row=row, col=col
        )

    # Calculate ylim based on data in xlim range
    mask = (E >= xlim[0]) & (E <= xlim[1])
    if np.any(mask):
        y_max = np.max((primary + secondary)[mask])
        ylim = (0, y_max * 1.05)  # Add 5% padding at the top
    else:
        ylim = (0, 1)  # Default if no data in range

    # Axis labels and limits - updated style
    fig.update_xaxes(
        range=xlim,
        title_text="Energy (keV)",
        showgrid=False,        # Remove grid lines
        showticklabels=True,   # Keep tick labels
        ticks="outside",       # Show tick marks outside the plot
        showline=True,         # Show axis line
        linewidth=1,           # Axis line thickness
        linecolor='black',     # Axis line color
        mirror=False,          # Don't mirror to top
        zeroline=False,
        row=row, col=col
    )
    fig.update_yaxes(
        range=ylim,
        title_text="Counts",
        showgrid=False,        # Remove grid lines
        showticklabels=True,   # Keep tick labels
        ticks="outside",       # Show tick marks outside the plot
        showline=True,         # Show axis line
        linewidth=1,           # Axis line thickness
        linecolor='black',     # Axis line color
        mirror=False,          # Don't mirror to right
        zeroline=False,
        row=row, col=col
    )

    # SNR annotation
    subtitle = (
        f"<b>Pb Lα</b> (10.5 keV): SNR = {data['alpha']['SNR']:.1f} "
        f"(Ns = {int(data['alpha']['secondary'])}, Nb = {int(data['alpha']['primary'])})<br>"
        f"<b>Pb Lβ</b> (12.6 keV): SNR = {data['beta']['SNR']:.1f} "
        f"(Ns = {int(data['beta']['secondary'])}, Nb = {int(data['beta']['primary'])})"
    )
    fig.add_annotation(
        x=0.98, y=0.98,
        xref=f"x{2*(row-1)+col} domain",
        yref=f"y{2*(row-1)+col} domain",
        text=subtitle,
        showarrow=False,
        font=dict(size=9),
        align="right",
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="gray",
        borderwidth=1
    )

# ============================================================
# ROW 2: FULL SPECTRA
# ============================================================

add_spectrum(
    row=2, col=1,
    E=E, primary=b["primary"], secondary=b["secondary"],
    peak=peak, xlim=(5, 70), data=b, showlegend=True
)

add_spectrum(
    row=2, col=2,
    E=E, primary=t["primary"], secondary=t["secondary"],
    peak=peak, xlim=(5, 70), data=t, showlegend=False
)

# ============================================================
# ROW 3: ZOOMED SPECTRA (Pb L-PEAKS)
# ============================================================

add_spectrum(
    row=3, col=1,
    E=E, primary=b["primary"], secondary=b["secondary"],
    peak=peak, xlim=(9.5, 13.5), data=b, showlegend=False
)

add_spectrum(
    row=3, col=2,
    E=E, primary=t["primary"], secondary=t["secondary"],
    peak=peak, xlim=(9.5, 13.5), data=t, showlegend=False
)

# ============================================================
# FINAL LAYOUT
# ============================================================

fig.update_layout(
    height=1100,
    width=1000,
    title={
        'text': f"<sup>100 ppm Pb | {exposure_hours}h exposure | "
                f"{detector_area_mm2} mm² detector | {FWHM_keV:.2f} keV FWHM</sup>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 16}
    },
    font=dict(size=11),
    margin=dict(t=100, l=60, r=40, b=60),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

Experimental parameters
Detector energy resolution (FWHM): 0.15 keV
Detector energy bin bidth: 40 eV (0.040 keV)
Detector area: 50 mm²
Exposure time: 10 hours


<IPython.core.display.Javascript object>


📊 Adding energy resolution...
🔍 Filtering events based on exposure time and detector area...
  - Backscatter: 103,150 events detected
  - Transmission: 11,829,478 events detected
📈 Creating energy histograms...
🎯 Calculating Signal-to-Noise Ratios...

Results summary

Backscatter geometry:
  Pb Lα: SNR = 3.7 (Signal: 21, Background: 11)
  Pb Lβ: SNR = 2.5 (Signal: 20, Background: 46)

Transmission geometry:
  Pb Lα: SNR = 8.1 (Signal: 66, Background: 1)
  Pb Lβ: SNR = 6.4 (Signal: 98, Background: 133)

✓ Processing complete!
🎨 Generating visualization...
